<a href="https://colab.research.google.com/github/ping-Mel/ERV-V2-Assignments/blob/main/session6/part2/s6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
       # Convolutional layers
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)  # Reduced filters to 8
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1) # Reduced filters to 16
        self.bn2 = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(2, 2)
        self.drop = nn.Dropout(0.25)

        self.conv3 = nn.Conv2d(16, 32, 3, padding=1) # Reduced filters to 32
        self.bn3 = nn.BatchNorm2d(32)
        # Fully connected layer
        # Adjust the number of input features to the FC layer based on the output size of the last conv layer
        self.fc1 = nn.Linear(32 * 7 * 7, 120)  # This needs adjustment
        self.fc2 = nn.Linear(120, 10)  # Final output layer

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.drop(x)
        x = F.relu(self.bn3(self.conv3(x)))

        # Flatten the output for the FC layer
        x = x.view(-1, 32 * 7 * 7)  # Adjust based on the actual output size
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [18]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
         MaxPool2d-3            [-1, 8, 14, 14]               0
            Conv2d-4           [-1, 16, 14, 14]           1,168
       BatchNorm2d-5           [-1, 16, 14, 14]              32
         MaxPool2d-6             [-1, 16, 7, 7]               0
           Dropout-7             [-1, 16, 7, 7]               0
            Conv2d-8             [-1, 32, 7, 7]           4,640
       BatchNorm2d-9             [-1, 32, 7, 7]              64
           Linear-10                  [-1, 120]         188,280
           Linear-11                   [-1, 10]           1,210
Total params: 195,490
Trainable params: 195,490
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/

In [19]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [20]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [21]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.0696355327963829 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.53it/s]



Test set: Average loss: 0.0652, Accuracy: 9797/10000 (97.97%)



loss=0.03696713596582413 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.20it/s]



Test set: Average loss: 0.0425, Accuracy: 9863/10000 (98.63%)



loss=0.03210648521780968 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 27.94it/s]


...


Test set: Average loss: 0.0193, Accuracy: 9942/10000 (99.42%)



loss=0.017073826864361763 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 27.59it/s]



Test set: Average loss: 0.0232, Accuracy: 9928/10000 (99.28%)



loss=0.017583219334483147 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.05it/s]



Test set: Average loss: 0.0219, Accuracy: 9937/10000 (99.37%)

